## Step 1 - Start local geth node


In [ ]:
# Step 1 - Start local geth node
# Run runGeth script    
# $ ./00_runGeth.sh
# 
# Or the following but remember to include the preloaded keyfiles for all actors 
# geth --dev --dev.period 1 --datadir ./testchain --rpc --rpccorsdomain '*' --rpcport 8646 --rpcapi "eth,net,web3,debug" --port 32323 --maxpeers 0 --targetgaslimit 994712388 console
#
# Step 2 - Run notebook
# $ ./01_notebook.sh

In [6]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# load web3 python helper functions 
import functions
import web3, solcx, json
from web3 import Web3
from web3.auto.gethdev import w3

from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Connect to local node
w3 = Web3(Web3.IPCProvider("./testchain/geth.ipc"))

In [ ]:
# Unlock user accounts
functions.unlockAccounts('');

## Step 2 - Check Environment

In [9]:
# Check that our geth node is running
block = w3.eth.getBlock('latest')
block.hash
block.number

46

In [10]:
block.hash


HexBytes('0xb1f8f583a0e9fc96e2b7daed53f80a52da520e1a7ef87788e4549b5efd0484aa')

In [11]:
# that our test accounts have loaded  
w3.eth.accounts[3]
# = '0xa33a6c312D9aD0E0F2E95541BeED0Cc081621fd0'

'0xa33a6c312D9aD0E0F2E95541BeED0Cc081621fd0'

In [17]:
# Test helper fucntions 
functions.printBalances()

 # Account                                      EtherBalanceChange                   Token Name
-- ------------------------------------------ -------------------- ----------------------- ---------------------
 1 0xa11AAE29840fBb5c86E6fd4cF809EBA183AEf433                    0                       0 Account #1 - Owner
 2 0xa22AB8A9D641CE77e06D98b7D7065d324D3d6976                    0                       0 Account #2 - Alice
 3 0xa33a6c312D9aD0E0F2E95541BeED0Cc081621fd0                    0                       0 Account #3 - Bob
 4 0xa44a08d3F6933c69212114bb66E2Df1813651844                    0                       0 Account #4 - Carol
 5 0xa55A151Eb00fded1634D27D1127b4bE4627079EA                    0                       0 Account #5 - Dave
 6 0xa66a85ede0CBE03694AA9d9dE0BB19c99ff55bD9                    0                       0 Account #6
 7 0xa77A2b9D4B1c010A22A7c565Dc418cef683DbceC                    0                       0 Account #7
 8 0xA88A05d2b88283ce84C8325760B72a64591

## Step 3 - Compile contracts

In [ ]:
# Move files to testing directory 

SOURCEDIR="../../contracts"
REDBLACKLIBSOL="BokkyPooBahsRedBlackTreeLibrary.sol"
TESTREDBLACKSOL="TestBokkyPooBahsRedBlackTree.sol"
REDBLACKLIBDIR = SOURCEDIR + REDBLACKLIBSOL

In [ ]:
!cp $SOURCEDIR/$REDBLACKLIBSOL . 
!cp $SOURCEDIR/$TESTREDBLACKSOL .

## Step 3 - Compile using solc (Option 1)

In [ ]:
RedBlackTreeLibraryOutput = !solc --optimize --combined-json abi,bin,interface {REDBLACKLIBSOL}

In [ ]:
redBlackTreeLibraryAbi = json.loads(RedBlackTreeLibraryOutput[0])['contracts'][REDBLACKLIBSOL + ':BokkyPooBahsRedBlackTreeLibrary']['abi'] 
redBlackTreeLibraryAbi

In [ ]:
redBlackTreeLibraryBin = json.loads(RedBlackTreeLibraryOutput[0])['contracts'][REDBLACKLIBSOL + ':BokkyPooBahsRedBlackTreeLibrary']['bin']
redBlackTreeLibraryBin

In [ ]:
# Build the contract object in preparation for deployment
token_deployer = w3.eth.contract(abi=redBlackTreeLibraryAbi, bytecode=redBlackTreeLibraryBin)

## Step 3 - Compile using Remix (Option 2)

In [ ]:
#Copied bytecode from Remix 

bytecode = {
	"linkReferences": {},
	"object": "608060405234801561001057600080fd5b50610028600064010000000061157761002d82021704565b6100aa565b600281015460ff161561003f57600080fd5b600080825560408051608081018252828152602080820184815282840185815260608401868152868052600180890190945294909520925183555182820155925160028083019190915591516003909101805491151560ff1992831617905592018054909216179055565b611620806100b96000396000f3006080604052600436106100e55763ffffffff7c010000000000000000000000000000000000000000000000000000000060003504166306661abd81146100ea5780633107ee4f1461011157806335671214146101265780633df4ddf41461013e57806347799da8146101535780634c818994146101685780634cc822151461017d5780634eaa9d22146101975780634f0f4aa9146101af5780634f558e79146101f457806365e921431461022057806383af31b91461023857806390b5561d14610250578063c0a4d38f14610268578063ebf0c71714610280578063edd004e514610295575b600080fd5b3480156100f657600080fd5b506100ff6102ad565b60408051918252519081900360200190f35b34801561011d57600080fd5b506100ff6102be565b34801561013257600080fd5b506100ff6004356102c4565b34801561014a57600080fd5b506100ff6102dc565b34801561015f57600080fd5b506100ff6102e8565b34801561017457600080fd5b506100ff6102f4565b34801561018957600080fd5b506101956004356102fa565b005b3480156101a357600080fd5b506100ff60043561030e565b3480156101bb57600080fd5b506101c7600435610320565b60408051958652602086019490945284840192909252606084015215156080830152519081900360a00190f35b34801561020057600080fd5b5061020c600435610348565b604080519115158252519081900360200190f35b34801561022c57600080fd5b506100ff60043561035a565b34801561024457600080fd5b506100ff60043561036c565b34801561025c57600080fd5b5061019560043561037e565b34801561027457600080fd5b506100ff60043561038f565b34801561028c57600080fd5b506100ff6103a1565b3480156102a157600080fd5b506100ff6004356103a7565b60006102b960006103b9565b905090565b60045490565b60006102d6818363ffffffff6103e416565b92915050565b60006102b96000610483565b60006102b960006104c5565b60035490565b61030b60008263ffffffff61050716565b50565b60006102d6818363ffffffff6107bb16565b600080808080610336818763ffffffff6107df16565b939a9299509097509550909350915050565b60006102d6818363ffffffff6108cb16565b60006102d6818363ffffffff61093716565b60006102d6818363ffffffff6109ba16565b61030b60008263ffffffff610a0c16565b60006102d6818363ffffffff610b4516565b60005490565b60006102d6818363ffffffff610b8416565b60008160040154826003015410156103d25760006102d6565b5060048101546003820154035b919050565b60008115156103f257600080fd5b60008281526001808501602052604090912001541561043157600082815260018085016020526040909120015461042a908490610c1c565b90506102d6565b5060008181526001830160205260409020545b80158015906104655750600081815260018085016020526040909120015482145b156102d6576000818152600184016020526040902054909150610444565b80545b80158015906104a75750600081815260018084016020526040909120015415155b156103df576000908152600180830160205260409091200154610486565b80545b80158015906104e95750600081815260018301602052604090206002015415155b156103df5760009081526001820160205260409020600201546104c8565b600080808084151561051857600080fd5b85548514806105425750855485148015906105425750600085815260018701602052604090205415155b151561054d57600080fd5b6000858152600180880160205260409091200154158061057e57506000858152600187016020526040902060020154155b1561058b578492506105d7565b600085815260018701602052604090206002015492505b6000838152600180880160205260409091200154156105d75760009283526001808701602052604090932090920154916105a2565b60008381526001808801602052604090912001541561060b5760008381526001808801602052604090912001549350610622565b600083815260018701602052604090206002015493505b60008381526001870160205260408082205486835291208190559150811561069657600082815260018088016020526040909120015483141561067a5760008281526001808801602052604090912001849055610691565b600082815260018701602052604090206002018490555b61069a565b8386555b50600082815260018601602052604090206003015460ff1615848314610725576106c5868487610c59565b60008581526001878101602052604080832080830154878552828520938401819055845281842087905560028082015490840181905584529083208690556003908101549286905201805460ff191660ff90921615159190911790559193915b8015610735576107358685610cd5565b6000808052600187016020526040902054156107795760008080526001808801602052604082208281559081018290556002810191909155600301805460ff191690555b5050600090815260018085016020526040822082815580820183905560028101929092556003909101805460ff19169055600490930180549093019092555050565b60008115156107c957600080fd5b5060009081526001909101602052604090205490565b6000806000806000806107f061154c565b8715156107fc57600080fd5b885491505b81156108ac578188141561086c575060008781526001808a0160209081526040928390208351608081018552815480825293820154928101839052600282015494810185905260039091015460ff161515606082018190528b9a5092985090965091945092506108bf565b818810156108905760009182526001808a01602052604090922090910154906108a7565b600091825260018901602052604090912060020154905b610801565b6000806000806000965096509650965096505b50509295509295909350565b6000808215156108da57600080fd5b5082545b801561093057808314156108f55760019150610930565b8083101561091657600090815260018085016020526040909120015461092b565b60009081526001840160205260409020600201545b6108de565b5092915050565b60008082151561094657600080fd5b50600082815260018401602052604090205480156109b057600081815260018086016020526040909120015483141561099457600081815260018501602052604090206002015491506109ab565b600081815260018086016020526040909120015491505b610930565b5060009392505050565b600080808315156109ca57600080fd5b6109d48585610b45565b915081156109ff575060008381526001850160205260409020546109f88582610937565b9250610a04565b600092505b505092915050565b60008080831515610a1c57600080fd5b505082546000915081905b8015610a7c5780915080841015610a51576000908152600180860160205260409091200154610a77565b80841415610a625760019250610a7c565b60009081526001850160205260409020600201545b610a27565b8215610a8757600080fd5b60408051608081018252838152600060208083018281528385018381526001606086018181528b86528c82019094529590932093518455519383019390935551600282015590516003909101805460ff1916911515919091179055811515610af157838555610b2b565b81841015610b145760008281526001808701602052604090912001849055610b2b565b600082815260018601602052604090206002018490555b610b3585856110f6565b5050505060030180546001019055565b600080821515610b5457600080fd5b50600082815260018401602052604090205480156109b05760008181526001850160205260409020549150610930565b6000811515610b9257600080fd5b600082815260018401602052604090206002015415610bca57600082815260018401602052604090206002015461042a908490611363565b5060008181526001830160205260409020545b8015801590610bfe5750600081815260018401602052604090206002015482145b156102d6576000818152600184016020526040902054909150610bdd565b60005b600082815260018401602052604090206002015415610c535760009182526001830160205260409091206002015490610c1f565b50919050565b6000818152600184016020526040808220548483529120819055801515610c8257828455610ccf565b6000818152600180860160205260409091200154821415610cb85760008181526001808601602052604090912001839055610ccf565b600081815260018501602052604090206002018390555b50505050565b6000805b83548314801590610cfe5750600083815260018501602052604090206003015460ff16155b156110d657506000828152600180850160205260408083205480845292200154831415610efe5760008181526001850160205260408082206002015480835291206003015490925060ff1615610da3576000828152600180860160205260408083206003908101805460ff19908116909155858552919093209092018054909216179055610d8c848261139b565b600081815260018501602052604090206002015491505b60008281526001808601602052604080832090910154825290206003015460ff16158015610dee5750600082815260018501602052604080822060020154825290206003015460ff16155b15610e1b57600082815260018581016020526040909120600301805460ff19169091179055915081610ef9565b600082815260018501602052604080822060020154825290206003015460ff161515610e9a576000828152600180860160205260408083208083015484529083206003908101805460ff1990811690915593869052018054909216179055610e838483611475565b600081815260018501602052604090206002015491505b600081815260018501602052604080822060039081018054868552838520808401805460ff909316151560ff199384161790558254821690925560029091015484529190922090910180549091169055610ef4848261139b565b835492505b6110d1565b6000818152600180860160205260408083209091015480835291206003015490925060ff1615610f7d576000828152600180860160205260408083206003908101805460ff19908116909155858552919093209092018054909216179055610f668482611475565b600081815260018086016020526040909120015491505b600082815260018501602052604080822060020154825290206003015460ff16158015610fc8575060008281526001808601602052604080832090910154825290206003015460ff16155b15610ff557600082815260018581016020526040909120600301805460ff191690911790559150816110d1565b60008281526001808601602052604080832090910154825290206003015460ff16151561107657600082815260018086016020526040808320600281015484529083206003908101805460ff199081169091559386905201805490921617905561105f848361139b565b600081815260018086016020526040909120015491505b60008181526001808601602052604080832060039081018054878652838620808401805460ff909316151560ff199384161790558254821690925593015484529220909101805490911690556110cc8482611475565b835492505b610cd9565b50506000908152600190910160205260409020600301805460ff19169055565b6000805b835483148015906111245750600083815260018501602052604080822054825290206003015460ff165b15611340575060008281526001808501602052604080832054808452818420548452922001548114156112485760008181526001850160205260408082205482528082206002015480835291206003015490925060ff16156111ce576000818152600180860160205260408083206003808201805460ff19908116909155878652838620820180548216905582548652928520018054909216909217905590829052549250611243565b60008181526001850160205260409020600201548314156111f6578092506111f6848461139b565b50600082815260018085016020526040808320548084528184206003808201805460ff19908116909155825487529386200180549093169093179091559182905254611243908590611475565b61133b565b6000818152600180860160205260408083205483528083209091015480835291206003015490925060ff16156112c6576000818152600180860160205260408083206003808201805460ff1990811690915587865283862082018054821690558254865292852001805490921690921790559082905254925061133b565b60008181526001808601602052604090912001548314156112ee578092506112ee8484611475565b50600082815260018085016020526040808320548084528184206003808201805460ff1990811690915582548752938620018054909316909317909155918290525461133b90859061139b565b6110fa565b50508154600090815260019092016020525060409020600301805460ff19169055565b60005b600082815260018085016020526040909120015415610c53576000918252600180840160205260409092209091015490611366565b6000818152600180840160205260408083206002810180549154828652928520909301549385905291839055909180156113e357600081815260018601602052604090208490555b6000838152600186016020526040902082905581151561140557828555611452565b600082815260018087016020526040909120015484141561143b5760008281526001808701602052604090912001839055611452565b600082815260018601602052604090206002018390555b505060008181526001938401602052604080822090940183905591825291902055565b600081815260018084016020526040808320918201805492548385529184206002015493859052839055909180156114bb57600081815260018601602052604090208490555b600083815260018601602052604090208290558115156114dd5782855561152a565b6000828152600186016020526040902060020154841415611513576000828152600186016020526040902060020183905561152a565b600082815260018087016020526040909120018390555b5050600081815260019093016020526040808420600201839055918352912055565b6080604051908101604052806000815260200160008152602001600081526020016000151581525090565b600281015460ff161561158957600080fd5b600080825560408051608081018252828152602080820184815282840185815260608401868152868052600180890190945294909520925183555182820155925160028083019190915591516003909101805491151560ff19928316179055920180549092161790555600a165627a7a7230582073d08353060aa43a58e4dcea735af27a3a50b27a5f4f5a02fdd00c055e3a28580029",
	"opcodes": "PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 ISZERO PUSH2 0x10 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0x28 PUSH1 0x0 PUSH5 0x100000000 PUSH2 0x1577 PUSH2 0x2D DUP3 MUL OR DIV JUMP JUMPDEST PUSH2 0xAA JUMP JUMPDEST PUSH1 0x2 DUP2 ADD SLOAD PUSH1 0xFF AND ISZERO PUSH2 0x3F JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST PUSH1 0x0 DUP1 DUP3 SSTORE PUSH1 0x40 DUP1 MLOAD PUSH1 0x80 DUP2 ADD DUP3 MSTORE DUP3 DUP2 MSTORE PUSH1 0x20 DUP1 DUP3 ADD DUP5 DUP2 MSTORE DUP3 DUP5 ADD DUP6 DUP2 MSTORE PUSH1 0x60 DUP5 ADD DUP7 DUP2 MSTORE DUP7 DUP1 MSTORE PUSH1 0x1 DUP1 DUP10 ADD SWAP1 SWAP5 MSTORE SWAP5 SWAP1 SWAP6 KECCAK256 SWAP3 MLOAD DUP4 SSTORE MLOAD DUP3 DUP3 ADD SSTORE SWAP3 MLOAD PUSH1 0x2 DUP1 DUP4 ADD SWAP2 SWAP1 SWAP2 SSTORE SWAP2 MLOAD PUSH1 0x3 SWAP1 SWAP2 ADD DUP1 SLOAD SWAP2 ISZERO ISZERO PUSH1 0xFF NOT SWAP3 DUP4 AND OR SWAP1 SSTORE SWAP3 ADD DUP1 SLOAD SWAP1 SWAP3 AND OR SWAP1 SSTORE JUMP JUMPDEST PUSH2 0x1620 DUP1 PUSH2 0xB9 PUSH1 0x0 CODECOPY PUSH1 0x0 RETURN STOP PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x4 CALLDATASIZE LT PUSH2 0xE5 JUMPI PUSH4 0xFFFFFFFF PUSH29 0x100000000000000000000000000000000000000000000000000000000 PUSH1 0x0 CALLDATALOAD DIV AND PUSH4 0x6661ABD DUP2 EQ PUSH2 0xEA JUMPI DUP1 PUSH4 0x3107EE4F EQ PUSH2 0x111 JUMPI DUP1 PUSH4 0x35671214 EQ PUSH2 0x126 JUMPI DUP1 PUSH4 0x3DF4DDF4 EQ PUSH2 0x13E JUMPI DUP1 PUSH4 0x47799DA8 EQ PUSH2 0x153 JUMPI DUP1 PUSH4 0x4C818994 EQ PUSH2 0x168 JUMPI DUP1 PUSH4 0x4CC82215 EQ PUSH2 0x17D JUMPI DUP1 PUSH4 0x4EAA9D22 EQ PUSH2 0x197 JUMPI DUP1 PUSH4 0x4F0F4AA9 EQ PUSH2 0x1AF JUMPI DUP1 PUSH4 0x4F558E79 EQ PUSH2 0x1F4 JUMPI DUP1 PUSH4 0x65E92143 EQ PUSH2 0x220 JUMPI DUP1 PUSH4 0x83AF31B9 EQ PUSH2 0x238 JUMPI DUP1 PUSH4 0x90B5561D EQ PUSH2 0x250 JUMPI DUP1 PUSH4 0xC0A4D38F EQ PUSH2 0x268 JUMPI DUP1 PUSH4 0xEBF0C717 EQ PUSH2 0x280 JUMPI DUP1 PUSH4 0xEDD004E5 EQ PUSH2 0x295 JUMPI JUMPDEST PUSH1 0x0 DUP1 REVERT JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0xF6 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH2 0x2AD JUMP JUMPDEST PUSH1 0x40 DUP1 MLOAD SWAP2 DUP3 MSTORE MLOAD SWAP1 DUP2 SWAP1 SUB PUSH1 0x20 ADD SWAP1 RETURN JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x11D JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH2 0x2BE JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x132 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH1 0x4 CALLDATALOAD PUSH2 0x2C4 JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x14A JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH2 0x2DC JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x15F JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH2 0x2E8 JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x174 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH2 0x2F4 JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x189 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0x195 PUSH1 0x4 CALLDATALOAD PUSH2 0x2FA JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x1A3 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH1 0x4 CALLDATALOAD PUSH2 0x30E JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x1BB JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0x1C7 PUSH1 0x4 CALLDATALOAD PUSH2 0x320 JUMP JUMPDEST PUSH1 0x40 DUP1 MLOAD SWAP6 DUP7 MSTORE PUSH1 0x20 DUP7 ADD SWAP5 SWAP1 SWAP5 MSTORE DUP5 DUP5 ADD SWAP3 SWAP1 SWAP3 MSTORE PUSH1 0x60 DUP5 ADD MSTORE ISZERO ISZERO PUSH1 0x80 DUP4 ADD MSTORE MLOAD SWAP1 DUP2 SWAP1 SUB PUSH1 0xA0 ADD SWAP1 RETURN JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x200 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0x20C PUSH1 0x4 CALLDATALOAD PUSH2 0x348 JUMP JUMPDEST PUSH1 0x40 DUP1 MLOAD SWAP2 ISZERO ISZERO DUP3 MSTORE MLOAD SWAP1 DUP2 SWAP1 SUB PUSH1 0x20 ADD SWAP1 RETURN JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x22C JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH1 0x4 CALLDATALOAD PUSH2 0x35A JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x244 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH1 0x4 CALLDATALOAD PUSH2 0x36C JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x25C JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0x195 PUSH1 0x4 CALLDATALOAD PUSH2 0x37E JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x274 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH1 0x4 CALLDATALOAD PUSH2 0x38F JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x28C JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH2 0x3A1 JUMP JUMPDEST CALLVALUE DUP1 ISZERO PUSH2 0x2A1 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0xFF PUSH1 0x4 CALLDATALOAD PUSH2 0x3A7 JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2B9 PUSH1 0x0 PUSH2 0x3B9 JUMP JUMPDEST SWAP1 POP SWAP1 JUMP JUMPDEST PUSH1 0x4 SLOAD SWAP1 JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2D6 DUP2 DUP4 PUSH4 0xFFFFFFFF PUSH2 0x3E4 AND JUMP JUMPDEST SWAP3 SWAP2 POP POP JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2B9 PUSH1 0x0 PUSH2 0x483 JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2B9 PUSH1 0x0 PUSH2 0x4C5 JUMP JUMPDEST PUSH1 0x3 SLOAD SWAP1 JUMP JUMPDEST PUSH2 0x30B PUSH1 0x0 DUP3 PUSH4 0xFFFFFFFF PUSH2 0x507 AND JUMP JUMPDEST POP JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2D6 DUP2 DUP4 PUSH4 0xFFFFFFFF PUSH2 0x7BB AND JUMP JUMPDEST PUSH1 0x0 DUP1 DUP1 DUP1 DUP1 PUSH2 0x336 DUP2 DUP8 PUSH4 0xFFFFFFFF PUSH2 0x7DF AND JUMP JUMPDEST SWAP4 SWAP11 SWAP3 SWAP10 POP SWAP1 SWAP8 POP SWAP6 POP SWAP1 SWAP4 POP SWAP2 POP POP JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2D6 DUP2 DUP4 PUSH4 0xFFFFFFFF PUSH2 0x8CB AND JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2D6 DUP2 DUP4 PUSH4 0xFFFFFFFF PUSH2 0x937 AND JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2D6 DUP2 DUP4 PUSH4 0xFFFFFFFF PUSH2 0x9BA AND JUMP JUMPDEST PUSH2 0x30B PUSH1 0x0 DUP3 PUSH4 0xFFFFFFFF PUSH2 0xA0C AND JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2D6 DUP2 DUP4 PUSH4 0xFFFFFFFF PUSH2 0xB45 AND JUMP JUMPDEST PUSH1 0x0 SLOAD SWAP1 JUMP JUMPDEST PUSH1 0x0 PUSH2 0x2D6 DUP2 DUP4 PUSH4 0xFFFFFFFF PUSH2 0xB84 AND JUMP JUMPDEST PUSH1 0x0 DUP2 PUSH1 0x4 ADD SLOAD DUP3 PUSH1 0x3 ADD SLOAD LT ISZERO PUSH2 0x3D2 JUMPI PUSH1 0x0 PUSH2 0x2D6 JUMP JUMPDEST POP PUSH1 0x4 DUP2 ADD SLOAD PUSH1 0x3 DUP3 ADD SLOAD SUB JUMPDEST SWAP2 SWAP1 POP JUMP JUMPDEST PUSH1 0x0 DUP2 ISZERO ISZERO PUSH2 0x3F2 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD ISZERO PUSH2 0x431 JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD PUSH2 0x42A SWAP1 DUP5 SWAP1 PUSH2 0xC1C JUMP JUMPDEST SWAP1 POP PUSH2 0x2D6 JUMP JUMPDEST POP PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP4 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD JUMPDEST DUP1 ISZERO DUP1 ISZERO SWAP1 PUSH2 0x465 JUMPI POP PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD DUP3 EQ JUMPDEST ISZERO PUSH2 0x2D6 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD SWAP1 SWAP2 POP PUSH2 0x444 JUMP JUMPDEST DUP1 SLOAD JUMPDEST DUP1 ISZERO DUP1 ISZERO SWAP1 PUSH2 0x4A7 JUMPI POP PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD ISZERO ISZERO JUMPDEST ISZERO PUSH2 0x3DF JUMPI PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 DUP1 DUP4 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD PUSH2 0x486 JUMP JUMPDEST DUP1 SLOAD JUMPDEST DUP1 ISZERO DUP1 ISZERO SWAP1 PUSH2 0x4E9 JUMPI POP PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP4 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD ISZERO ISZERO JUMPDEST ISZERO PUSH2 0x3DF JUMPI PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 DUP3 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD PUSH2 0x4C8 JUMP JUMPDEST PUSH1 0x0 DUP1 DUP1 DUP1 DUP5 ISZERO ISZERO PUSH2 0x518 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST DUP6 SLOAD DUP6 EQ DUP1 PUSH2 0x542 JUMPI POP DUP6 SLOAD DUP6 EQ DUP1 ISZERO SWAP1 PUSH2 0x542 JUMPI POP PUSH1 0x0 DUP6 DUP2 MSTORE PUSH1 0x1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD ISZERO ISZERO JUMPDEST ISZERO ISZERO PUSH2 0x54D JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST PUSH1 0x0 DUP6 DUP2 MSTORE PUSH1 0x1 DUP1 DUP9 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD ISZERO DUP1 PUSH2 0x57E JUMPI POP PUSH1 0x0 DUP6 DUP2 MSTORE PUSH1 0x1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD ISZERO JUMPDEST ISZERO PUSH2 0x58B JUMPI DUP5 SWAP3 POP PUSH2 0x5D7 JUMP JUMPDEST PUSH1 0x0 DUP6 DUP2 MSTORE PUSH1 0x1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD SWAP3 POP JUMPDEST PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP1 DUP9 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD ISZERO PUSH2 0x5D7 JUMPI PUSH1 0x0 SWAP3 DUP4 MSTORE PUSH1 0x1 DUP1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP4 KECCAK256 SWAP1 SWAP3 ADD SLOAD SWAP2 PUSH2 0x5A2 JUMP JUMPDEST PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP1 DUP9 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD ISZERO PUSH2 0x60B JUMPI PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP1 DUP9 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD SWAP4 POP PUSH2 0x622 JUMP JUMPDEST PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD SWAP4 POP JUMPDEST PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 SLOAD DUP7 DUP4 MSTORE SWAP2 KECCAK256 DUP2 SWAP1 SSTORE SWAP2 POP DUP2 ISZERO PUSH2 0x696 JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP9 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD DUP4 EQ ISZERO PUSH2 0x67A JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP9 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD DUP5 SWAP1 SSTORE PUSH2 0x691 JUMP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD DUP5 SWAP1 SSTORE JUMPDEST PUSH2 0x69A JUMP JUMPDEST DUP4 DUP7 SSTORE JUMPDEST POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND ISZERO DUP5 DUP4 EQ PUSH2 0x725 JUMPI PUSH2 0x6C5 DUP7 DUP5 DUP8 PUSH2 0xC59 JUMP JUMPDEST PUSH1 0x0 DUP6 DUP2 MSTORE PUSH1 0x1 DUP8 DUP2 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 DUP1 DUP4 ADD SLOAD DUP8 DUP6 MSTORE DUP3 DUP6 KECCAK256 SWAP4 DUP5 ADD DUP2 SWAP1 SSTORE DUP5 MSTORE DUP2 DUP5 KECCAK256 DUP8 SWAP1 SSTORE PUSH1 0x2 DUP1 DUP3 ADD SLOAD SWAP1 DUP5 ADD DUP2 SWAP1 SSTORE DUP5 MSTORE SWAP1 DUP4 KECCAK256 DUP7 SWAP1 SSTORE PUSH1 0x3 SWAP1 DUP2 ADD SLOAD SWAP3 DUP7 SWAP1 MSTORE ADD DUP1 SLOAD PUSH1 0xFF NOT AND PUSH1 0xFF SWAP1 SWAP3 AND ISZERO ISZERO SWAP2 SWAP1 SWAP2 OR SWAP1 SSTORE SWAP2 SWAP4 SWAP2 JUMPDEST DUP1 ISZERO PUSH2 0x735 JUMPI PUSH2 0x735 DUP7 DUP6 PUSH2 0xCD5 JUMP JUMPDEST PUSH1 0x0 DUP1 DUP1 MSTORE PUSH1 0x1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD ISZERO PUSH2 0x779 JUMPI PUSH1 0x0 DUP1 DUP1 MSTORE PUSH1 0x1 DUP1 DUP9 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP3 KECCAK256 DUP3 DUP2 SSTORE SWAP1 DUP2 ADD DUP3 SWAP1 SSTORE PUSH1 0x2 DUP2 ADD SWAP2 SWAP1 SWAP2 SSTORE PUSH1 0x3 ADD DUP1 SLOAD PUSH1 0xFF NOT AND SWAP1 SSTORE JUMPDEST POP POP PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP3 KECCAK256 DUP3 DUP2 SSTORE DUP1 DUP3 ADD DUP4 SWAP1 SSTORE PUSH1 0x2 DUP2 ADD SWAP3 SWAP1 SWAP3 SSTORE PUSH1 0x3 SWAP1 SWAP2 ADD DUP1 SLOAD PUSH1 0xFF NOT AND SWAP1 SSTORE PUSH1 0x4 SWAP1 SWAP4 ADD DUP1 SLOAD SWAP1 SWAP4 ADD SWAP1 SWAP3 SSTORE POP POP JUMP JUMPDEST PUSH1 0x0 DUP2 ISZERO ISZERO PUSH2 0x7C9 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 SWAP1 SWAP2 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD SWAP1 JUMP JUMPDEST PUSH1 0x0 DUP1 PUSH1 0x0 DUP1 PUSH1 0x0 DUP1 PUSH2 0x7F0 PUSH2 0x154C JUMP JUMPDEST DUP8 ISZERO ISZERO PUSH2 0x7FC JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST DUP9 SLOAD SWAP2 POP JUMPDEST DUP2 ISZERO PUSH2 0x8AC JUMPI DUP2 DUP9 EQ ISZERO PUSH2 0x86C JUMPI POP PUSH1 0x0 DUP8 DUP2 MSTORE PUSH1 0x1 DUP1 DUP11 ADD PUSH1 0x20 SWAP1 DUP2 MSTORE PUSH1 0x40 SWAP3 DUP4 SWAP1 KECCAK256 DUP4 MLOAD PUSH1 0x80 DUP2 ADD DUP6 MSTORE DUP2 SLOAD DUP1 DUP3 MSTORE SWAP4 DUP3 ADD SLOAD SWAP3 DUP2 ADD DUP4 SWAP1 MSTORE PUSH1 0x2 DUP3 ADD SLOAD SWAP5 DUP2 ADD DUP6 SWAP1 MSTORE PUSH1 0x3 SWAP1 SWAP2 ADD SLOAD PUSH1 0xFF AND ISZERO ISZERO PUSH1 0x60 DUP3 ADD DUP2 SWAP1 MSTORE DUP12 SWAP11 POP SWAP3 SWAP9 POP SWAP1 SWAP7 POP SWAP2 SWAP5 POP SWAP3 POP PUSH2 0x8BF JUMP JUMPDEST DUP2 DUP9 LT ISZERO PUSH2 0x890 JUMPI PUSH1 0x0 SWAP2 DUP3 MSTORE PUSH1 0x1 DUP1 DUP11 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP3 KECCAK256 SWAP1 SWAP2 ADD SLOAD SWAP1 PUSH2 0x8A7 JUMP JUMPDEST PUSH1 0x0 SWAP2 DUP3 MSTORE PUSH1 0x1 DUP10 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 PUSH1 0x2 ADD SLOAD SWAP1 JUMPDEST PUSH2 0x801 JUMP JUMPDEST PUSH1 0x0 DUP1 PUSH1 0x0 DUP1 PUSH1 0x0 SWAP7 POP SWAP7 POP SWAP7 POP SWAP7 POP SWAP7 POP JUMPDEST POP POP SWAP3 SWAP6 POP SWAP3 SWAP6 SWAP1 SWAP4 POP JUMP JUMPDEST PUSH1 0x0 DUP1 DUP3 ISZERO ISZERO PUSH2 0x8DA JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP DUP3 SLOAD JUMPDEST DUP1 ISZERO PUSH2 0x930 JUMPI DUP1 DUP4 EQ ISZERO PUSH2 0x8F5 JUMPI PUSH1 0x1 SWAP2 POP PUSH2 0x930 JUMP JUMPDEST DUP1 DUP4 LT ISZERO PUSH2 0x916 JUMPI PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD PUSH2 0x92B JUMP JUMPDEST PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD JUMPDEST PUSH2 0x8DE JUMP JUMPDEST POP SWAP3 SWAP2 POP POP JUMP JUMPDEST PUSH1 0x0 DUP1 DUP3 ISZERO ISZERO PUSH2 0x946 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD DUP1 ISZERO PUSH2 0x9B0 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD DUP4 EQ ISZERO PUSH2 0x994 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD SWAP2 POP PUSH2 0x9AB JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD SWAP2 POP JUMPDEST PUSH2 0x930 JUMP JUMPDEST POP PUSH1 0x0 SWAP4 SWAP3 POP POP POP JUMP JUMPDEST PUSH1 0x0 DUP1 DUP1 DUP4 ISZERO ISZERO PUSH2 0x9CA JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST PUSH2 0x9D4 DUP6 DUP6 PUSH2 0xB45 JUMP JUMPDEST SWAP2 POP DUP2 ISZERO PUSH2 0x9FF JUMPI POP PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD PUSH2 0x9F8 DUP6 DUP3 PUSH2 0x937 JUMP JUMPDEST SWAP3 POP PUSH2 0xA04 JUMP JUMPDEST PUSH1 0x0 SWAP3 POP JUMPDEST POP POP SWAP3 SWAP2 POP POP JUMP JUMPDEST PUSH1 0x0 DUP1 DUP1 DUP4 ISZERO ISZERO PUSH2 0xA1C JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP POP DUP3 SLOAD PUSH1 0x0 SWAP2 POP DUP2 SWAP1 JUMPDEST DUP1 ISZERO PUSH2 0xA7C JUMPI DUP1 SWAP2 POP DUP1 DUP5 LT ISZERO PUSH2 0xA51 JUMPI PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD PUSH2 0xA77 JUMP JUMPDEST DUP1 DUP5 EQ ISZERO PUSH2 0xA62 JUMPI PUSH1 0x1 SWAP3 POP PUSH2 0xA7C JUMP JUMPDEST PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD JUMPDEST PUSH2 0xA27 JUMP JUMPDEST DUP3 ISZERO PUSH2 0xA87 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST PUSH1 0x40 DUP1 MLOAD PUSH1 0x80 DUP2 ADD DUP3 MSTORE DUP4 DUP2 MSTORE PUSH1 0x0 PUSH1 0x20 DUP1 DUP4 ADD DUP3 DUP2 MSTORE DUP4 DUP6 ADD DUP4 DUP2 MSTORE PUSH1 0x1 PUSH1 0x60 DUP7 ADD DUP2 DUP2 MSTORE DUP12 DUP7 MSTORE DUP13 DUP3 ADD SWAP1 SWAP5 MSTORE SWAP6 SWAP1 SWAP4 KECCAK256 SWAP4 MLOAD DUP5 SSTORE MLOAD SWAP4 DUP4 ADD SWAP4 SWAP1 SWAP4 SSTORE MLOAD PUSH1 0x2 DUP3 ADD SSTORE SWAP1 MLOAD PUSH1 0x3 SWAP1 SWAP2 ADD DUP1 SLOAD PUSH1 0xFF NOT AND SWAP2 ISZERO ISZERO SWAP2 SWAP1 SWAP2 OR SWAP1 SSTORE DUP2 ISZERO ISZERO PUSH2 0xAF1 JUMPI DUP4 DUP6 SSTORE PUSH2 0xB2B JUMP JUMPDEST DUP2 DUP5 LT ISZERO PUSH2 0xB14 JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD DUP5 SWAP1 SSTORE PUSH2 0xB2B JUMP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD DUP5 SWAP1 SSTORE JUMPDEST PUSH2 0xB35 DUP6 DUP6 PUSH2 0x10F6 JUMP JUMPDEST POP POP POP POP PUSH1 0x3 ADD DUP1 SLOAD PUSH1 0x1 ADD SWAP1 SSTORE JUMP JUMPDEST PUSH1 0x0 DUP1 DUP3 ISZERO ISZERO PUSH2 0xB54 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD DUP1 ISZERO PUSH2 0x9B0 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD SWAP2 POP PUSH2 0x930 JUMP JUMPDEST PUSH1 0x0 DUP2 ISZERO ISZERO PUSH2 0xB92 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD ISZERO PUSH2 0xBCA JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD PUSH2 0x42A SWAP1 DUP5 SWAP1 PUSH2 0x1363 JUMP JUMPDEST POP PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP4 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD JUMPDEST DUP1 ISZERO DUP1 ISZERO SWAP1 PUSH2 0xBFE JUMPI POP PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD DUP3 EQ JUMPDEST ISZERO PUSH2 0x2D6 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 SLOAD SWAP1 SWAP2 POP PUSH2 0xBDD JUMP JUMPDEST PUSH1 0x0 JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD ISZERO PUSH2 0xC53 JUMPI PUSH1 0x0 SWAP2 DUP3 MSTORE PUSH1 0x1 DUP4 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 PUSH1 0x2 ADD SLOAD SWAP1 PUSH2 0xC1F JUMP JUMPDEST POP SWAP2 SWAP1 POP JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 SLOAD DUP5 DUP4 MSTORE SWAP2 KECCAK256 DUP2 SWAP1 SSTORE DUP1 ISZERO ISZERO PUSH2 0xC82 JUMPI DUP3 DUP5 SSTORE PUSH2 0xCCF JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD DUP3 EQ ISZERO PUSH2 0xCB8 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD DUP4 SWAP1 SSTORE PUSH2 0xCCF JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD DUP4 SWAP1 SSTORE JUMPDEST POP POP POP POP JUMP JUMPDEST PUSH1 0x0 DUP1 JUMPDEST DUP4 SLOAD DUP4 EQ DUP1 ISZERO SWAP1 PUSH2 0xCFE JUMPI POP PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND ISZERO JUMPDEST ISZERO PUSH2 0x10D6 JUMPI POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SLOAD DUP1 DUP5 MSTORE SWAP3 KECCAK256 ADD SLOAD DUP4 EQ ISZERO PUSH2 0xEFE JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 PUSH1 0x2 ADD SLOAD DUP1 DUP4 MSTORE SWAP2 KECCAK256 PUSH1 0x3 ADD SLOAD SWAP1 SWAP3 POP PUSH1 0xFF AND ISZERO PUSH2 0xDA3 JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 PUSH1 0x3 SWAP1 DUP2 ADD DUP1 SLOAD PUSH1 0xFF NOT SWAP1 DUP2 AND SWAP1 SWAP2 SSTORE DUP6 DUP6 MSTORE SWAP2 SWAP1 SWAP4 KECCAK256 SWAP1 SWAP3 ADD DUP1 SLOAD SWAP1 SWAP3 AND OR SWAP1 SSTORE PUSH2 0xD8C DUP5 DUP3 PUSH2 0x139B JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD SWAP2 POP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SWAP1 SWAP2 ADD SLOAD DUP3 MSTORE SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND ISZERO DUP1 ISZERO PUSH2 0xDEE JUMPI POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 PUSH1 0x2 ADD SLOAD DUP3 MSTORE SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND ISZERO JUMPDEST ISZERO PUSH2 0xE1B JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP6 DUP2 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 PUSH1 0x3 ADD DUP1 SLOAD PUSH1 0xFF NOT AND SWAP1 SWAP2 OR SWAP1 SSTORE SWAP2 POP DUP2 PUSH2 0xEF9 JUMP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 PUSH1 0x2 ADD SLOAD DUP3 MSTORE SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND ISZERO ISZERO PUSH2 0xE9A JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 DUP1 DUP4 ADD SLOAD DUP5 MSTORE SWAP1 DUP4 KECCAK256 PUSH1 0x3 SWAP1 DUP2 ADD DUP1 SLOAD PUSH1 0xFF NOT SWAP1 DUP2 AND SWAP1 SWAP2 SSTORE SWAP4 DUP7 SWAP1 MSTORE ADD DUP1 SLOAD SWAP1 SWAP3 AND OR SWAP1 SSTORE PUSH2 0xE83 DUP5 DUP4 PUSH2 0x1475 JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD SWAP2 POP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 PUSH1 0x3 SWAP1 DUP2 ADD DUP1 SLOAD DUP7 DUP6 MSTORE DUP4 DUP6 KECCAK256 DUP1 DUP5 ADD DUP1 SLOAD PUSH1 0xFF SWAP1 SWAP4 AND ISZERO ISZERO PUSH1 0xFF NOT SWAP4 DUP5 AND OR SWAP1 SSTORE DUP3 SLOAD DUP3 AND SWAP1 SWAP3 SSTORE PUSH1 0x2 SWAP1 SWAP2 ADD SLOAD DUP5 MSTORE SWAP2 SWAP1 SWAP3 KECCAK256 SWAP1 SWAP2 ADD DUP1 SLOAD SWAP1 SWAP2 AND SWAP1 SSTORE PUSH2 0xEF4 DUP5 DUP3 PUSH2 0x139B JUMP JUMPDEST DUP4 SLOAD SWAP3 POP JUMPDEST PUSH2 0x10D1 JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SWAP1 SWAP2 ADD SLOAD DUP1 DUP4 MSTORE SWAP2 KECCAK256 PUSH1 0x3 ADD SLOAD SWAP1 SWAP3 POP PUSH1 0xFF AND ISZERO PUSH2 0xF7D JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 PUSH1 0x3 SWAP1 DUP2 ADD DUP1 SLOAD PUSH1 0xFF NOT SWAP1 DUP2 AND SWAP1 SWAP2 SSTORE DUP6 DUP6 MSTORE SWAP2 SWAP1 SWAP4 KECCAK256 SWAP1 SWAP3 ADD DUP1 SLOAD SWAP1 SWAP3 AND OR SWAP1 SSTORE PUSH2 0xF66 DUP5 DUP3 PUSH2 0x1475 JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD SWAP2 POP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 PUSH1 0x2 ADD SLOAD DUP3 MSTORE SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND ISZERO DUP1 ISZERO PUSH2 0xFC8 JUMPI POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SWAP1 SWAP2 ADD SLOAD DUP3 MSTORE SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND ISZERO JUMPDEST ISZERO PUSH2 0xFF5 JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP6 DUP2 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 PUSH1 0x3 ADD DUP1 SLOAD PUSH1 0xFF NOT AND SWAP1 SWAP2 OR SWAP1 SSTORE SWAP2 POP DUP2 PUSH2 0x10D1 JUMP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SWAP1 SWAP2 ADD SLOAD DUP3 MSTORE SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND ISZERO ISZERO PUSH2 0x1076 JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 PUSH1 0x2 DUP2 ADD SLOAD DUP5 MSTORE SWAP1 DUP4 KECCAK256 PUSH1 0x3 SWAP1 DUP2 ADD DUP1 SLOAD PUSH1 0xFF NOT SWAP1 DUP2 AND SWAP1 SWAP2 SSTORE SWAP4 DUP7 SWAP1 MSTORE ADD DUP1 SLOAD SWAP1 SWAP3 AND OR SWAP1 SSTORE PUSH2 0x105F DUP5 DUP4 PUSH2 0x139B JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD SWAP2 POP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 PUSH1 0x3 SWAP1 DUP2 ADD DUP1 SLOAD DUP8 DUP7 MSTORE DUP4 DUP7 KECCAK256 DUP1 DUP5 ADD DUP1 SLOAD PUSH1 0xFF SWAP1 SWAP4 AND ISZERO ISZERO PUSH1 0xFF NOT SWAP4 DUP5 AND OR SWAP1 SSTORE DUP3 SLOAD DUP3 AND SWAP1 SWAP3 SSTORE SWAP4 ADD SLOAD DUP5 MSTORE SWAP3 KECCAK256 SWAP1 SWAP2 ADD DUP1 SLOAD SWAP1 SWAP2 AND SWAP1 SSTORE PUSH2 0x10CC DUP5 DUP3 PUSH2 0x1475 JUMP JUMPDEST DUP4 SLOAD SWAP3 POP JUMPDEST PUSH2 0xCD9 JUMP JUMPDEST POP POP PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 SWAP1 SWAP2 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x3 ADD DUP1 SLOAD PUSH1 0xFF NOT AND SWAP1 SSTORE JUMP JUMPDEST PUSH1 0x0 DUP1 JUMPDEST DUP4 SLOAD DUP4 EQ DUP1 ISZERO SWAP1 PUSH2 0x1124 JUMPI POP PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 SLOAD DUP3 MSTORE SWAP1 KECCAK256 PUSH1 0x3 ADD SLOAD PUSH1 0xFF AND JUMPDEST ISZERO PUSH2 0x1340 JUMPI POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SLOAD DUP1 DUP5 MSTORE DUP2 DUP5 KECCAK256 SLOAD DUP5 MSTORE SWAP3 KECCAK256 ADD SLOAD DUP2 EQ ISZERO PUSH2 0x1248 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 SLOAD DUP3 MSTORE DUP1 DUP3 KECCAK256 PUSH1 0x2 ADD SLOAD DUP1 DUP4 MSTORE SWAP2 KECCAK256 PUSH1 0x3 ADD SLOAD SWAP1 SWAP3 POP PUSH1 0xFF AND ISZERO PUSH2 0x11CE JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 PUSH1 0x3 DUP1 DUP3 ADD DUP1 SLOAD PUSH1 0xFF NOT SWAP1 DUP2 AND SWAP1 SWAP2 SSTORE DUP8 DUP7 MSTORE DUP4 DUP7 KECCAK256 DUP3 ADD DUP1 SLOAD DUP3 AND SWAP1 SSTORE DUP3 SLOAD DUP7 MSTORE SWAP3 DUP6 KECCAK256 ADD DUP1 SLOAD SWAP1 SWAP3 AND SWAP1 SWAP3 OR SWAP1 SSTORE SWAP1 DUP3 SWAP1 MSTORE SLOAD SWAP3 POP PUSH2 0x1243 JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD DUP4 EQ ISZERO PUSH2 0x11F6 JUMPI DUP1 SWAP3 POP PUSH2 0x11F6 DUP5 DUP5 PUSH2 0x139B JUMP JUMPDEST POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SLOAD DUP1 DUP5 MSTORE DUP2 DUP5 KECCAK256 PUSH1 0x3 DUP1 DUP3 ADD DUP1 SLOAD PUSH1 0xFF NOT SWAP1 DUP2 AND SWAP1 SWAP2 SSTORE DUP3 SLOAD DUP8 MSTORE SWAP4 DUP7 KECCAK256 ADD DUP1 SLOAD SWAP1 SWAP4 AND SWAP1 SWAP4 OR SWAP1 SWAP2 SSTORE SWAP2 DUP3 SWAP1 MSTORE SLOAD PUSH2 0x1243 SWAP1 DUP6 SWAP1 PUSH2 0x1475 JUMP JUMPDEST PUSH2 0x133B JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SLOAD DUP4 MSTORE DUP1 DUP4 KECCAK256 SWAP1 SWAP2 ADD SLOAD DUP1 DUP4 MSTORE SWAP2 KECCAK256 PUSH1 0x3 ADD SLOAD SWAP1 SWAP3 POP PUSH1 0xFF AND ISZERO PUSH2 0x12C6 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 PUSH1 0x3 DUP1 DUP3 ADD DUP1 SLOAD PUSH1 0xFF NOT SWAP1 DUP2 AND SWAP1 SWAP2 SSTORE DUP8 DUP7 MSTORE DUP4 DUP7 KECCAK256 DUP3 ADD DUP1 SLOAD DUP3 AND SWAP1 SSTORE DUP3 SLOAD DUP7 MSTORE SWAP3 DUP6 KECCAK256 ADD DUP1 SLOAD SWAP1 SWAP3 AND SWAP1 SWAP3 OR SWAP1 SSTORE SWAP1 DUP3 SWAP1 MSTORE SLOAD SWAP3 POP PUSH2 0x133B JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD DUP4 EQ ISZERO PUSH2 0x12EE JUMPI DUP1 SWAP3 POP PUSH2 0x12EE DUP5 DUP5 PUSH2 0x1475 JUMP JUMPDEST POP PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SLOAD DUP1 DUP5 MSTORE DUP2 DUP5 KECCAK256 PUSH1 0x3 DUP1 DUP3 ADD DUP1 SLOAD PUSH1 0xFF NOT SWAP1 DUP2 AND SWAP1 SWAP2 SSTORE DUP3 SLOAD DUP8 MSTORE SWAP4 DUP7 KECCAK256 ADD DUP1 SLOAD SWAP1 SWAP4 AND SWAP1 SWAP4 OR SWAP1 SWAP2 SSTORE SWAP2 DUP3 SWAP1 MSTORE SLOAD PUSH2 0x133B SWAP1 DUP6 SWAP1 PUSH2 0x139B JUMP JUMPDEST PUSH2 0x10FA JUMP JUMPDEST POP POP DUP2 SLOAD PUSH1 0x0 SWAP1 DUP2 MSTORE PUSH1 0x1 SWAP1 SWAP3 ADD PUSH1 0x20 MSTORE POP PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x3 ADD DUP1 SLOAD PUSH1 0xFF NOT AND SWAP1 SSTORE JUMP JUMPDEST PUSH1 0x0 JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP6 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD ISZERO PUSH2 0xC53 JUMPI PUSH1 0x0 SWAP2 DUP3 MSTORE PUSH1 0x1 DUP1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP3 KECCAK256 SWAP1 SWAP2 ADD SLOAD SWAP1 PUSH2 0x1366 JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 PUSH1 0x2 DUP2 ADD DUP1 SLOAD SWAP2 SLOAD DUP3 DUP7 MSTORE SWAP3 DUP6 KECCAK256 SWAP1 SWAP4 ADD SLOAD SWAP4 DUP6 SWAP1 MSTORE SWAP2 DUP4 SWAP1 SSTORE SWAP1 SWAP2 DUP1 ISZERO PUSH2 0x13E3 JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 DUP5 SWAP1 SSTORE JUMPDEST PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 DUP3 SWAP1 SSTORE DUP2 ISZERO ISZERO PUSH2 0x1405 JUMPI DUP3 DUP6 SSTORE PUSH2 0x1452 JUMP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD SLOAD DUP5 EQ ISZERO PUSH2 0x143B JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD DUP4 SWAP1 SSTORE PUSH2 0x1452 JUMP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD DUP4 SWAP1 SSTORE JUMPDEST POP POP PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 SWAP4 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP3 KECCAK256 SWAP1 SWAP5 ADD DUP4 SWAP1 SSTORE SWAP2 DUP3 MSTORE SWAP2 SWAP1 KECCAK256 SSTORE JUMP JUMPDEST PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP1 DUP5 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP4 KECCAK256 SWAP2 DUP3 ADD DUP1 SLOAD SWAP3 SLOAD DUP4 DUP6 MSTORE SWAP2 DUP5 KECCAK256 PUSH1 0x2 ADD SLOAD SWAP4 DUP6 SWAP1 MSTORE DUP4 SWAP1 SSTORE SWAP1 SWAP2 DUP1 ISZERO PUSH2 0x14BB JUMPI PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 DUP5 SWAP1 SSTORE JUMPDEST PUSH1 0x0 DUP4 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 DUP3 SWAP1 SSTORE DUP2 ISZERO ISZERO PUSH2 0x14DD JUMPI DUP3 DUP6 SSTORE PUSH2 0x152A JUMP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD SLOAD DUP5 EQ ISZERO PUSH2 0x1513 JUMPI PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP7 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 KECCAK256 PUSH1 0x2 ADD DUP4 SWAP1 SSTORE PUSH2 0x152A JUMP JUMPDEST PUSH1 0x0 DUP3 DUP2 MSTORE PUSH1 0x1 DUP1 DUP8 ADD PUSH1 0x20 MSTORE PUSH1 0x40 SWAP1 SWAP2 KECCAK256 ADD DUP4 SWAP1 SSTORE JUMPDEST POP POP PUSH1 0x0 DUP2 DUP2 MSTORE PUSH1 0x1 SWAP1 SWAP4 ADD PUSH1 0x20 MSTORE PUSH1 0x40 DUP1 DUP5 KECCAK256 PUSH1 0x2 ADD DUP4 SWAP1 SSTORE SWAP2 DUP4 MSTORE SWAP2 KECCAK256 SSTORE JUMP JUMPDEST PUSH1 0x80 PUSH1 0x40 MLOAD SWAP1 DUP2 ADD PUSH1 0x40 MSTORE DUP1 PUSH1 0x0 DUP2 MSTORE PUSH1 0x20 ADD PUSH1 0x0 DUP2 MSTORE PUSH1 0x20 ADD PUSH1 0x0 DUP2 MSTORE PUSH1 0x20 ADD PUSH1 0x0 ISZERO ISZERO DUP2 MSTORE POP SWAP1 JUMP JUMPDEST PUSH1 0x2 DUP2 ADD SLOAD PUSH1 0xFF AND ISZERO PUSH2 0x1589 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST PUSH1 0x0 DUP1 DUP3 SSTORE PUSH1 0x40 DUP1 MLOAD PUSH1 0x80 DUP2 ADD DUP3 MSTORE DUP3 DUP2 MSTORE PUSH1 0x20 DUP1 DUP3 ADD DUP5 DUP2 MSTORE DUP3 DUP5 ADD DUP6 DUP2 MSTORE PUSH1 0x60 DUP5 ADD DUP7 DUP2 MSTORE DUP7 DUP1 MSTORE PUSH1 0x1 DUP1 DUP10 ADD SWAP1 SWAP5 MSTORE SWAP5 SWAP1 SWAP6 KECCAK256 SWAP3 MLOAD DUP4 SSTORE MLOAD DUP3 DUP3 ADD SSTORE SWAP3 MLOAD PUSH1 0x2 DUP1 DUP4 ADD SWAP2 SWAP1 SWAP2 SSTORE SWAP2 MLOAD PUSH1 0x3 SWAP1 SWAP2 ADD DUP1 SLOAD SWAP2 ISZERO ISZERO PUSH1 0xFF NOT SWAP3 DUP4 AND OR SWAP1 SSTORE SWAP3 ADD DUP1 SLOAD SWAP1 SWAP3 AND OR SWAP1 SSTORE JUMP STOP LOG1 PUSH6 0x627A7A723058 KECCAK256 PUSH20 0xD08353060AA43A58E4DCEA735AF27A3A50B27A5F 0x4f GAS MUL REVERT 0xd0 0xc SDIV 0x5e GASPRICE 0x28 PC STOP 0x29 ",
	"sourceMap": "15676:1983:0:-;;;15957:49;8:9:-1;5:2;;;30:1;27;20:12;5:2;-1:-1;15988:11:0;:4;:9;;;;;;:11;:::i;:::-;15676:1983;;1081:222;1142:16;;;;;;1141:17;1133:26;;;;;;972:1;1169:20;;;1222:41;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;1199:20;;;1222:41;1199:10;;;:20;;;;;;;:64;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;-1:-1:-1;;1199:64:0;;;;;;1273:16;;:23;;;;;;;;1081:222::o;15676:1983::-;;;;;;;"
}


In [ ]:
token_code = bytecode['object']

In [ ]:
# token_code

In [ ]:
# Build the contract object in preparation for deployment
token_deployer = w3.eth.contract(abi=token_abi, bytecode=token_code)


## Step 4 - Build transaction


In [ ]:
init = token_deployer.constructor()
txn = init.buildTransaction({'gas': 320000})

In [ ]:
txn

In [ ]:
./testchain

In [ ]:
# define
def sign_transaction(txn):
    # Flesh out the transaction for local signing
    next_nonce = w3.eth.getTransactionCount(str(w3.eth.accounts[0]))
    signable_transaction = dict(
      txn,
      nonce=next_nonce,
      gasPrice=w3.toWei(4, 'gwei'),
    )
    # Sign transaction
    encrypted_key = open('testchain/keystore/UTC--2017-05-20T02-37-30.360937280Z--a00af22d07c87d96eeeb0ed583f8f6ac7812827e').read()
    private_key = w3.eth.account.decrypt(encrypted_key, '')
    signature_info = w3.eth.account.signTransaction(signable_transaction, private_key)
    # Broadcast transaction
    txn_hash = w3.eth.sendRawTransaction(signature_info.rawTransaction)
    # Wait for the transaction to be mined
    receipt = w3.eth.waitForTransactionReceipt(txn_hash)

    return receipt

In [ ]:
# Test Code
receipt = sign_transaction(txn)
receipt

## Step 5 - Interact with Contract

In [ ]:
## Get deployed contract
#token = w3.eth.contract(address=receipt.contractAddress, abi=token_abi)

## Alternatively, deploy it in remix and use the address
Contractaddress = Web3.toChecksumAddress(0xfd08772a704f00702d7f983c3ebecdec9d16814b)
token = w3.eth.contract(address=Contractaddress, abi=token_abi)


In [ ]:
# Check the count
token.functions.count().call()

In [ ]:
txn = token.functions.insert(11).buildTransaction({'gas': 320000})
txn

In [ ]:
# Test Code
receipt = sign_transaction(txn)
receipt

In [ ]:
def insertToList(insert_list):
    i = 0
    while (i < len(insert_list)):
        txn = token.functions.insert(insert_list[i]).buildTransaction({'gas': 320000})
        receipt = sign_transaction(txn)
        # print(receipt)
        print(i)
        i = i + 1

In [ ]:
insert_list = [67, 23]
insertToList(insert_list)

In [ ]:
root = token.functions.root().call()
root

In [ ]:
tree = token
nodeData = tree.functions.getNode(7).call()
nodeData

In [ ]:
def treeAsList(tree):
    result = []
    treeAsListWalker(tree, tree.functions.root().call(), result)
    return result

In [ ]:
def treeAsListWalker(tree, node, result):
    SENTINEL = 0
    nodeData = tree.functions.getNode(node).call()
    leftNode = nodeData[2]
    rightNode = nodeData[3]
    if (leftNode != SENTINEL):
        treeAsListWalker(tree, leftNode, result)
    if (nodeData[0] != SENTINEL):
        result.append(nodeData[0])
    if (rightNode != SENTINEL):
        treeAsListWalker(tree, rightNode, result)


In [ ]:
treeList = treeAsList(token)
print(treeList)

In [ ]:
result = []
root = token.functions.root().call()
treeAsListWalker(token, root, result)
print(result)

In [ ]:
insert_list = [14]
insertToList(insert_list)

In [ ]:
result = []
root = token.functions.root().call()
treeAsListWalker(token, root, result)
print(result)

In [ ]:
root = token.functions.root().call()
root